### This is another web scrapping related project. I got it via telegram. The client want to collect the data from a website and use that data in his own woo-commerce website. So, I just helped him.

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import warnings
warnings.filterwarnings('ignore')
import time
import re

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [97]:
pages=np.arange(1,6,1)

In [98]:
pages

array([1, 2, 3, 4, 5])

In [3]:
# url='https://sigsauerarmory.com/rifles/page/'

In [99]:
names = []
images = []
prices = []
stocks = []
categories = []
description=[]
specifications=[]
full_description=[]


In [100]:
for page in pages:
    page = requests.get("https://sigsauerarmory.com/rifles/page/"+str(page)+"/")
    # soup = BeautifulSoup(page.text, 'html.parser')
    soup=BeautifulSoup(page.content,'html.parser')
    pages=soup.find_all('h2')
    href_links = []
    for h2_tag in pages:
        link = h2_tag.find('a')['href'] if h2_tag.find('a') else None
        if link:
            href_links.append(link)

    unique_list = list(set(href_links))
    
    for i in unique_list:
        response = requests.get(i)
        if response.status_code == 200:
            soup_1 = BeautifulSoup(response.content, 'html.parser')
            n=soup_1.find('h1',class_='product_title entry-title')
            name = n.get_text()
            names.append(name)

            im=soup_1.find_all('img',class_='wp-post-image')
            src_attributes = [img['src'] for img in im]
            image=src_attributes[0]
            images.append(image)

            p=soup_1.find('bdi')
            # price_1=p.get_text()
            # price = re.search(r'\d+\.\d+', price_1).group()
            prices.append(p)

            s=soup_1.find('p',class_='stock in-stock')
            stock_1=s.get_text()
            

            stock = stock_1.split(':')[-1].strip()
            stocks.append(stock)

            c=soup_1.find('span',class_='posted_in')
            category_1=c.get_text()
            category=category_1.split(':')[-1].strip()
            categories.append(category)

            # d=soup_1.find_all('div',class_='col-sm-8 col-md-7')
            # description.append(d)

            # f=soup_1.find_all('div',class_='col-xs-12 box-outline')
            # specifications.append(f)

            a_1=soup_1.find('div',id='tab-description')
            full_description.append(a_1)


            time.sleep(20)
        else:
            print("Failed to retrieve content from", i)
    


In [41]:
# page = requests.get("https://sigsauerarmory.com/rifles/page/2/")
# # soup = BeautifulSoup(page.text, 'html.parser')
# soup=BeautifulSoup(page.content,'html.parser')

In [7]:
# pages=soup.find_all('h2')
# href_links = []
# for h2_tag in pages:
#     link = h2_tag.find('a')['href'] if h2_tag.find('a') else None
#     if link:
#         href_links.append(link)

# unique_list = list(set(href_links))

In [101]:
data = {
    'Name': names,
    'Image': images,
    'Price': prices,
    'Stock': stocks,
    'Category': categories,
    # 'description':description,
    # 'specifications':specifications,
    'full_description': full_description
}

df = pd.DataFrame(data)

In [103]:
df.to_csv(r'D:/Telegram/gennaro/rifles.csv',index=False)

In [109]:
df['Stock'] = df['Stock'].apply(lambda x: 1 if x == 'In stock' else 0)

In [113]:
df.duplicated().sum()

0

In [108]:
for i in df['full_description']:
    print(i)

<div aria-labelledby="tab-title-description" class="mf-Tabs-panel mf-Tabs-panel--description panel entry-content" id="tab-description" role="tabpanel">
<h3 class="tab-title">Description</h3>
<div class="tab-content-wrapper">
<div class="row padding-v-10">
<div class="col-xs-12 box-outline">
<div class="row">
<div class="col-xs-12 padding-v-10">
<div class="row padding-v-10">
<div class="col-sm-8 col-md-7">
<p>One of the most tested and proven weapons systems in the world achieves ultimate refinement with the introduction of the Sig Sauer SIG516 rifle. With its chrome-lined, phosphated, military grade barrel and adjustable, four-position gas regulator, the SIG516 incorporates all of the most desirable mechanical improvements. Add to these a free-floating quad rail forend and barrel, and you have the most advanced, versatile AR-based rifle available.</p>
<p>This California compliant model features compliant bullet button magazine release and 10 round magazine. Stock and grip may vary fro

In [116]:
df2=df.copy()

In [118]:
for i in df2['full_description']:
    print(i)

<div aria-labelledby="tab-title-description" class="mf-Tabs-panel mf-Tabs-panel--description panel entry-content" id="tab-description" role="tabpanel">
<h3 class="tab-title">Description</h3>
<div class="tab-content-wrapper">
<div class="row padding-v-10">
<div class="col-xs-12 box-outline">
<div class="row">
<div class="col-xs-12 padding-v-10">
<div class="row padding-v-10">
<div class="col-sm-8 col-md-7">
<p>One of the most tested and proven weapons systems in the world achieves ultimate refinement with the introduction of the Sig Sauer SIG516 rifle. With its chrome-lined, phosphated, military grade barrel and adjustable, four-position gas regulator, the SIG516 incorporates all of the most desirable mechanical improvements. Add to these a free-floating quad rail forend and barrel, and you have the most advanced, versatile AR-based rifle available.</p>
<p>This California compliant model features compliant bullet button magazine release and 10 round magazine. Stock and grip may vary fro

In [115]:
df3.to_csv(r'D:/Telegram/gennaro/new_test_4.csv',index=False)

In [120]:
description=[i for i in df2['full_description']]

In [126]:
description_1=str(description)

In [122]:
pattern = r'class="mf-Tabs-panel mf-Tabs-panel--description panel entry-content"'


In [144]:
def split_html(html_data):
    soup_2 = BeautifulSoup(html_data, 'html.parser')
    class_to_split = 'woocommerce-Tabs-panel woocommerce-Tabs-panel--description panel entry-content wc-tab'
    elements = soup_2.find_all('div',class_=class_to_split)
    
    if elements:
        return 'code'
    else:
        return 'none'



def split_html_1(html_data):
    soup_3 = BeautifulSoup(html_data, 'html.parser')
    class_to_split = 'mf-Tabs-panel mf-Tabs-panel--description panel entry-content'
    elements = soup_3.find_all('div',class_=class_to_split)
    
    if elements:
        return 'code'
    else:
        return 'none'

In [148]:
df2['html_data'] = df2['full_description'].apply(lambda x: x.to_string(escape=False))

TypeError: 'NoneType' object is not callable